In [1]:
from sympy import *
import numpy as np

In [69]:
u = lambda x, t : exp((-1*(pi)**2)*t)*sin(pi*x)
x=Symbol('x')
t=Symbol('t')
u(x,t).evalf(subs={x:0.2,t:0.5})
u(x,t).evalf(subs={x:1,t:0})

-0.e-101

In [163]:
def forward_diff(u, range_x, range_t, h, k, alpha, log=True):
    """
    numerical solution using forward difference method for parabolic heat equation
    """

    x=Symbol('x')
    t=Symbol('t')
    if log:
        print("u(x,t) = ",u(x,t))
        print("h=",h, ";x=",range_x[1])
        print("k=",k, ";t=",range_t[1])
        print("alpha=",alpha)
        
    l = (alpha**2)*k/(h**2)
    max_iter_i = int((range_x[1]-range_x[0])/h) + 1
    max_iter_j = int((range_t[1]-range_t[0])/k) + 1
    m = max_iter_i-1
    #initiate tridiagonal matrix A 
    A = np.zeros((m-1, m-1))
    for i in range(m-1):
        for j in range(m-1):
            if i==0:
                A[i] = np.array( [1-(2*l), l]+([0]*(len(A)-2)) )
            elif i==len(A)-1:
                A[i] = np.array( ([0]*(len(A)-2))+[l, 1-(2*l)] )
            else:
                A[i][i-1] = l
                A[i][i] = (1-(2*l))
                A[i][i+1] = l
    #insert zeros for u(0,t) and u(l,t)
    #A=np.insert(A, 0, np.zeros(m-1), 0)
    #A=np.insert(A, len(A), np.zeros(m-1), 0)
    if log:
        print("A = ")
        print(A)

    #initiate vector x input
    xi=np.array([h*i for i in range(max_iter_i)])

    #initiate w with size (m+1) * (m+1) (2 rows for u(0,t) and u(l,t))
    #w = np.zeros((max_iter_i, max_iter_j))
    w = np.zeros((m-1, max_iter_j))
    
    #initiate vector w at j=0, where 1<=i<=m-1, where w(i,0) = f(xi) 
    w10 = np.array([u(x,t).evalf(subs={x:xi[i], t:0}) for i in range(1, m)])
    
    #add zeros at both 0 and l
    #w10=np.insert(w10, 0, 0, 0)
    #w10=np.insert(w10, len(w10), 0, 0)
    
    #set it as w(1), but transpose w first, then insert w(1,0)
    w = np.transpose(w)
    w[0] = w10
    #w = np.transpose(w)
    #print(np.transpose(w))
    
    #calculate the w at j>0 until j*k = t
    for j in range(1, len(w)):
        w[j] = np.matmul(A, np.transpose(w[j-1]))
    if log :
        print("w(",len(w)-1,") = ")
        print(w[-1])
    return w[-1]

In [177]:
#Example 1 page 727
print("Example 1 page 727")
u = lambda x, t : exp((-1*(pi)**2)*t)*sin(pi*x)

#variables initialization
alpha = 1
k = 0.0005
h = 0.1
rn_x = [0,1]
rn_t = [0,0.5]
x_iter = int((rn_x[1]-rn_x[0])/h)
xi = np.array([h*i for i in range(1, x_iter)])

#exact calculation of u(x,t)
u_exact = np.array([u(x,t).evalf(subs={x:xi[i], t:rn_t[1]}) for i in range(len(xi))])

#approximation of u(x,t)
w_last = forward_diff(u, rn_x, rn_t, h, k, alpha)

print("\nResult :")
print("u(x,0.5)\tw(i,1000)\terror")
for i in range(len(w_last)):
    err = abs(u_exact[i] - w_last[i])
    print(u_exact[i],"\t",w_last[i],"\t",err)

Example 1 page 727
u(x,t) =  exp(-pi**2*t)*sin(pi*x)
h= 0.1 ;x= 1
k= 0.0005 ;t= 0.5
alpha= 1
A = 
[[0.9  0.05 0.   0.   0.   0.   0.   0.   0.  ]
 [0.05 0.9  0.05 0.   0.   0.   0.   0.   0.  ]
 [0.   0.05 0.9  0.05 0.   0.   0.   0.   0.  ]
 [0.   0.   0.05 0.9  0.05 0.   0.   0.   0.  ]
 [0.   0.   0.   0.05 0.9  0.05 0.   0.   0.  ]
 [0.   0.   0.   0.   0.05 0.9  0.05 0.   0.  ]
 [0.   0.   0.   0.   0.   0.05 0.9  0.05 0.  ]
 [0.   0.   0.   0.   0.   0.   0.05 0.9  0.05]
 [0.   0.   0.   0.   0.   0.   0.   0.05 0.9 ]]
w( 1000 ) = 
[0.00228652 0.00434922 0.00598619 0.00703719 0.00739934 0.00703719
 0.00598619 0.00434922 0.00228652]

Result :
u(x,0.5)	w(i,1000)	error
0.00222241417851267 	 0.0022865207865784528 	 6.41066080657787e-5
0.00422728297276244 	 0.004349220987439515 	 0.000121938014677076
0.00581835585642586 	 0.005986189135245531 	 0.000167833278819674
0.00683988752999332 	 0.007037187382261514 	 0.000197299852268192
0.00719188335582637 	 0.007399336697334161 	 0.00020745